In [1]:
import os
import requests
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
import time
import io
import csv
import os

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # driver = webdriver.Chrome(service=Service("/Users/mac/Downloads/chromedriver-mac-arm64/chromedriver"), options=options)
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1920, 1080)
    return driver

/Users/rosydai/Desktop/Master Project/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def handle_privacy_popup(driver):
    """
    Handles the privacy popup for Google.
    """
    try:
        reject_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., '全部接受')]"))
        )
        reject_button.click()
        print("Privacy popup dismissed.")
        time.sleep(2)
    except TimeoutException:
        print("No privacy popup found or already handled.")

In [31]:
# Initialize search terms
search_entity = {
    "game": ["Roblox", "Minecraft", "Among Us", "Clash Royale", "Pokémon Go"],
    "cartoon": ["Gravity Falls", "The Owl House", "Dexter's Laboratory", "SpongeBob SquarePants", "Powerpuff Girls"],
    "disney": ["Inside Out", "Elemental", "The Little Mermaid", "Turning Red", "Pirates of the Caribbean"],
    "toy": ["Barbie", "Pokemon card", "Lego", "Hot Wheels", "Rubik’s Cube"]
}

search_keywords = {
    "violence": ["torture", "murder", "violence", "Bullying", "fights", "weapons", "gunshot", "assault"],
    "sexual content": ["sex", "nude", "fuck", "porn", "vulgar", "dick", "nsfw", "explicit"],
    "shock and disgusting": ["horror", "horrible", "creepy", "creepiness", "dark", "bloody", "decapitation", "dismemberment"]
}
# Function to generate search terms
def generate_search_terms(search_entity, search_keywords):
    search_terms = []
    for entity_type, entities in search_entity.items():
        for entity in entities:
            for keyword_type, keywords in search_keywords.items():
                for keyword in keywords:
                    search_terms.append({
                        "entity_type": entity_type,
                        "entity": entity,
                        "keyword_type": keyword_type,
                        "keyword": keyword,
                        "search_term": f"{entity} {keyword}"
                    })
    return search_terms

In [32]:
print(len(generate_search_terms(search_entity, search_keywords)))
generate_search_terms(search_entity, search_keywords)[:10]

480


[{'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'torture',
  'search_term': 'Roblox torture'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'murder',
  'search_term': 'Roblox murder'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'violence',
  'search_term': 'Roblox violence'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'Bullying',
  'search_term': 'Roblox Bullying'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'fights',
  'search_term': 'Roblox fights'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'weapons',
  'search_term': 'Roblox weapons'},
 {'entity_type': 'game',
  'entity': 'Roblox',
  'keyword_type': 'violence',
  'keyword': 'gunshot',
  'search_term': 'Roblox gunshot'},
 {'entity_type': 'game',
  'entity': 'Rob

In [33]:
# Function to scrape images and save data
def scrape_images_and_save_to_csv(search_terms, output_csv, max_images=12):
    driver = init_driver()  # Initialize Selenium driver

    with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # Write the header
        writer.writerow(["Entity Type", "Entity", "Harmful Type", "Harmful Term", "Search Term", "File Name", "Image URL", "X", "Y", "Width", "Height"])

        for term in search_terms:
            entity_type = term["entity_type"]
            entity = term["entity"]
            harmful_type = term["keyword_type"]
            harmful_term = term["keyword"]
            search_term = term["search_term"]


            # Perform search
            driver.get(f"https://www.kiddle.co/i.php?q={search_term}#gsc.tab=1&gsc.q={search_term}&gsc.page=1")
            handle_privacy_popup(driver)
            time.sleep(5)

            # Scrape images
            images_found  = False
            try:
                # scrape images from kiddle website by css_selector, from first page, first three rows, each row 4 images
                images = driver.find_elements(By.CSS_SELECTOR, "img.gs-image.gs-image-scalable")[:max_images]
                # handle search terms with which no image result was found 
                if len(images) == 0:
                    writer.writerow([entity_type, entity, harmful_type, harmful_term, search_term, "NA", "NA", "NA", "NA", "NA"])
                else: 
                    for index, image in enumerate(images):
                        image_url = image.get_attribute("src") or image.get_attribute("data-src")
                        if not image_url:
                            continue
    
                        # Get image properties
                        x = image.location["x"]
                        y = image.location["y"]
                        width = image.size["width"]
                        height = image.size["height"]
                        file_name = f"{search_term.replace(' ', '_')}_image_{index + 1}"
    
                        # Save image data to CSV
                        writer.writerow([entity_type, entity, harmful_type, harmful_term, search_term,file_name, image_url, x, y, width, height])

                    
            except Exception as e:
                print(f"Error scraping {search_term}: {e}")
            

    driver.quit()
    print(f"Scraping completed. Data saved to {output_csv}")

# Main execution
if __name__ == "__main__":
    search_terms = generate_search_terms(search_entity, search_keywords)
    output_csv = "kiddle_scraped_data.csv"
    scrape_images_and_save_to_csv(search_terms, output_csv)

No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy popup found or already handled.
No privacy 